In [1]:
import pandas as pd
import joblib

In [2]:
df = pd.read_excel("data/tweet_bersih_100.xlsx")
twet = df.tweet.tolist()
label = df.label.tolist()
df.head()

,Unnamed: 0,tweet,label
0,0,buka visi misi begitu pukau sehingga buat pres...,positive
1,1,lebih baik pakai teknologi lama kaya negara in...,positive
2,2,kok arguing without crying kokarguing arguingw...,positive
3,3,jalan politik politik satu bukan politik pecah...,positive
4,4,juga fans lihat tampil serang hahaha jugafans ...,negative


In [3]:
pd.value_counts(df['label']).plot.bar(title='Data Sentimen')
print(pd.value_counts(df['label']))

negative    500
positive    500
Name: label, dtype: int64


# Pembagian data & pemodelan

In [4]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn import svm

In [5]:
# clf = svm.SVC(kernel = "rbf", C = 1, gamma=1)
# clf.fit(X, y)  

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = twet
vectorizer = TfidfVectorizer()
save = vectorizer.fit(corpus)
joblib.dump(save,"model/tdidf.w")
X = vectorizer.fit_transform(corpus)

In [7]:
k = 10
y = np.array(label)
kf = KFold(n_splits=k)
kf.get_n_splits(X)
# print(kf)
C_list     = np.arange(0.1, 2+0.1, 0.1)
gamma_list = np.arange(0.1, 2+0.1, 0.1)

C_list_ = list()
gamma_list_ = list()
rerata = list()
ix=1
for C in C_list:
    for gamma in gamma_list:
        C_list_.append(C)
        gamma_list_.append(gamma)
        
        clf = svm.SVC(kernel = "rbf", C = C, gamma=gamma)
        sc_10 = list()
        for train_index, test_index in kf.split(X):
        #     print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)
    #         clf.fit(X, y)
            sc = clf.score(X_test, y_test)
            sc_10.append(sc)
            
        if ix%100==0:
            print(ix)
        else:
            print(".", end="")
        ix+=1
            
        rerata.append(sum(sc_10)/k)
dict_ = {
    "C":C_list_,
    "gamma":gamma_list_,
    "score":rerata
}
df_hasil = pd.DataFrame.from_dict(dict_)
sorted_ = df_hasil.sort_values("score", axis = 0, ascending = False)
sorted_.head(10)

...................................................................................................100
...................................................................................................200
...................................................................................................300
...................................................................................................400


,C,gamma,score
384,2.0,0.5,0.765
383,2.0,0.4,0.764
385,2.0,0.6,0.764
364,1.9,0.5,0.763
365,1.9,0.6,0.762
345,1.8,0.6,0.761
325,1.7,0.6,0.760
344,1.8,0.5,0.760
386,2.0,0.7,0.759
326,1.7,0.7,0.759


In [14]:
len(C_list_)

400

In [15]:
sorted_.gamma.tolist()[0]

0.5

In [16]:
clf = clf = svm.SVC(kernel = "rbf", C = sorted_.C.tolist()[0], gamma=sorted_.gamma.tolist()[0])
save = clf.fit(X_train, y_train)
joblib.dump(save,"model/svm.m")

['model/svm.m']

In [17]:
save.score(X, y) #0.9761668321747765

0.958

In [12]:
x = save.predict(X)

In [13]:
ixx = 0
labelA = list()
labelP= list()

for i, j in zip(label, x):
    if i !=j:
        print(ixx)
        print(j)
        print("label asli",i)
        print(twet[ixx])
        print()
    ixx+=1

830
positive
label asli negative
pak all out debat malam semua umpan lawan lahap habis bahkan jawab kerap jadi serang balik pakall allout outdebat debatmalam malamsemua semuaumpan umpanlawan lawanlahap lahaphabis habisbahkan bahkanjawab jawabkerap kerapjadi jadiserang serangbalik pakallout alloutdebat outdebatmalam debatmalamsemua malamsemuaumpan semuaumpanlawan umpanlawanlahap lawanlahaphabis lahaphabisbahkan habisbahkanjawab bahkanjawabkerap jawabkerapjadi kerapjadiserang jadiserangbalik pakalloutdebat alloutdebatmalam outdebatmalamsemua debatmalamsemuaumpan malamsemuaumpanlawan semuaumpanlawanlahap umpanlawanlahaphabis lawanlahaphabisbahkan lahaphabisbahkanjawab habisbahkanjawabkerap bahkanjawabkerapjadi jawabkerapjadiserang kerapjadiserangbalik pakalloutdebatmalam alloutdebatmalamsemua outdebatmalamsemuaumpan debatmalamsemuaumpanlawan malamsemuaumpanlawanlahap semuaumpanlawanlahaphabis umpanlawanlahaphabisbahkan lawanlahaphabisbahkanjawab lahaphabisbahkanjawabkerap habisbahkanjawab